# Part 1: Automatic Speech Recognition, Diarize and Label

Environment = "whisperx"

* Performance Benchmarks on local
* GPU Benchmark: 0.09961056709289551 seconds
* Memory Bandwidth Benchmark: 0.2920224666595459 seconds
* CPU Benchmark: 13.046526432037354 seconds
* Disk Write Benchmark: 2.3364615440368652 seconds
* Disk Read Benchmark: 0.05882525444030762 seconds \n
  
** all benchmarks are >> faster than Collab with the exception of Disk write.

## Setup ⚙️
Tested for PyTorch 2.0, Python 3.10 (use other versions at your own risk!)
GPU execution requires the NVIDIA libraries cuBLAS 11.x and cuDNN 8.x to be installed on the system. Please refer to the CTranslate2 documentation.

1.  Create Python3.10 environment

`conda create --name whisperx python=3.10`

`conda activate whisperx`

2. Install PyTorch, e.g. for Linux and Windows CUDA11.8:
   
conda install pytorch==2.0.0 torchaudio==2.0.0 pytorch-cuda=11.8 -c pytorch -c nvidia

See other methods here.

1. Install this repo

`pip install git+https://github.com/m-bain/whisperx.git`

If already installed, update package to most recent commit

`pip install git+https://github.com/m-bain/whisperx.git --upgrade`



# Preprocess initial audio file
convert to Wav using ffmpeg

In [1]:
import ffmpeg

## 1 - Convert Mp3 to WAV.

def convert_m4a_to_mp3(input_file, output_file):
    try:
        ffmpeg.input(input_file).output(output_file).run(overwrite_output=True)
        print(f"Successfully converted {input_file} to {output_file}")
    except ffmpeg.Error as e:
        print("An error occurred:", e)

# Input/ output files and usage
input_mp3 = './audio/Botswana_2024_Audio.mp3'  # Change this to your mp3 file path
output_wav = './data/Botswana_2024_Audio.wav'  # Change this to your desired output wav file path

convert_m4a_to_mp3(input_mp3, output_wav)

Successfully converted ./audio/Botswana_2024_Audio.mp3 to ./data/Botswana_2024_Audio.wav


In [ ]:
import whisperx
import gc
import os
import torch

device = "cuda"
## Full file should be the input (2007 or 2024 file..)
audio_file = "./data/Botswana_2007_Audio.wav"


batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)
without_timestamps= 'True'

## Some error handling to ensure that successfully loaded the mp3 file!
try:
    # Check if the file exists
    if not os.path.isfile(audio_file):
        raise FileNotFoundError(f"The file '{audio_file}' does not exist.")
    # Optionally, you can add more checks (like file format) here

    print(f"Successfully accessed the audio file: {audio_file}")

except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

## Load the Audio File

In [ ]:
import whisperx
import gc
import os
import torch

device = "cuda"
## Full file should be the input (2007 or 2024 file..)
audio_file = "./audio/Botswana_2007_Audio.wav"

## DEBUGGING, use a small file
# audio_file = "./audio/Intro.wav"

batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)
without_timestamps= 'True'

## Some error handling to ensure that successfully loaded the mp3 file!
try:
    # Check if the file exists
    if not os.path.isfile(audio_file):
        raise FileNotFoundError(f"The file '{audio_file}' does not exist.")
    # Optionally, you can add more checks (like file format) here

    print(f"Successfully accessed the audio file: {audio_file}")

except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# BATCH PROCESS: Transcript - Align - Diarize

### Whisperx in the Terminal
This seemed to work on a single file VERY fast! 

In [ ]:
import os
from HF_token import TOKEN_ID
# Set the path to your directory
directory = "./audio/"

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".wav"):  # Check for .wav files
        filepath = os.path.join(directory, filename)
        
        # Construct and run the whisperx command for each file
        command = f"whisperx {filepath} --model large-v2 --diarize --highlight_words True --hf_token {TOKEN_ID} --output_dir ./outputs"
        os.system(command)

# Consolidate the Diarized JSON files 

In [2]:
import os
import pandas as pd
import json
import glob

# Directory containing the JSON files
json_directory = 'outputs/2024/'

# Get a list of all JSON files in the directory
json_files = glob.glob(os.path.join(json_directory, '*.json'))

# Initialize a list to hold all DataFrames
df_list = []

# Iterate through each JSON file and merge segments
for json_file in json_files:
	with open(json_file, 'r') as file:
		data = json.load(file)
		# Convert the "segments" part of the JSON data to a DataFrame
		df = pd.DataFrame(data["segments"])
		df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
diarized_df = pd.concat(df_list, ignore_index=True)

# Export
diarized_df.to_csv('./data/04.diarzed_output_no_names_2024.csv')

# Display the consolidated DataFrame
diarized_df.head(100)



,start,end,text,words,speaker
0,63.240,66.422,Hello and welcome to our last ever Grand Tour.,"[{'word': 'Hello', 'start': 63.24, 'end': 63.4...",SPEAKER_01
1,67.182,74.707,And because this is the last time that James R...,"[{'word': 'And', 'start': 67.182, 'end': 67.28...",SPEAKER_01
2,75.507,84.852,He said we should each buy a medium-sized elec...,"[{'word': 'He', 'start': 75.507, 'end': 75.687...",SPEAKER_01
3,86.393,89.415,And we decided to ignore him and come here.,"[{'word': 'And', 'start': 86.393, 'end': 86.47...",SPEAKER_01
4,104.378,107.662,"Yes, so welcome everyone to Zimbabwe.","[{'word': 'Yes,', 'start': 104.378, 'end': 104...",SPEAKER_03
...,...,...,...,...,...
95,385.276,387.599,I do like a Capri.,"[{'word': 'I', 'start': 385.276, 'end': 385.29...",SPEAKER_01
96,387.779,388.419,You've got to like it.,"[{'word': 'You've', 'start': 387.779, 'end': 3...",SPEAKER_01
97,388.459,389.500,It's the law.,"[{'word': 'It's', 'start': 388.459, 'end': 388...",SPEAKER_01
98,389.761,390.802,Everybody likes a Capri.,"[{'word': 'Everybody', 'start': 389.761, 'end'...",SPEAKER_01


## Apply Speaker Labels - 2007

In [7]:
# Apply the Speaker names to the labels

'''
Speaker 00 = Narrator
Speaker 01 = Hammond
Speaker 02 = Hammond
Speaker 03 = May
Speaker 04 = Clarkson
Speaker 05 = Clarkson
06 = Hammond
07 = Hammond
08 = Hammond
09 = ?

'''

diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_00', 'Narrator')
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_01', 'Hammond')
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_02', 'Hammond')

diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_03', 'May')
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_04', 'Clarkson')
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_05', 'Clarkson')

diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_06', 'Hammond')
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_07', 'Hammond')
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_08', 'Hammond')

diarized_df.head(100)

# Export
diarized_df.to_csv('./data/01.diarzed_output_named.csv')

## Apply Speaker labels 2024

  *NOTE* Version 1 of 2024 Diarize did not split speakers.Whisper model was unable to discern difference between James & Jeremy. I am sure one of those two would have something to say about that! 

In [ ]:
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_00', 'Narrator')
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_01', 'Clarkson')
diarized_df['speaker'] = diarized_df['speaker'].replace('SPEAKER_02', 'Hammond')


diarized_df.head(100)

# Export
diarized_df.to_csv('./data/05.diarzed_output_named_2024.csv')

# Part 2: LDA (Latent Dirichlet Allocation) Preparation




### Import the diarized data from (created previously)

In [6]:

import pandas as pd
import json

# Import
diarized_df = pd.read_csv('./data/04.diarzed_output_no_names_2024.csv')


In [7]:
preprocessed_df = diarized_df.copy()
# Preprocessing steps for LDA analysis

preprocessed_df

,Unnamed: 0,start,end,text,words,speaker
0,0,63.240,66.422,Hello and welcome to our last ever Grand Tour.,"[{'word': 'Hello', 'start': 63.24, 'end': 63.4...",SPEAKER_01
1,1,67.182,74.707,And because this is the last time that James R...,"[{'word': 'And', 'start': 67.182, 'end': 67.28...",SPEAKER_01
2,2,75.507,84.852,He said we should each buy a medium-sized elec...,"[{'word': 'He', 'start': 75.507, 'end': 75.687...",SPEAKER_01
3,3,86.393,89.415,And we decided to ignore him and come here.,"[{'word': 'And', 'start': 86.393, 'end': 86.47...",SPEAKER_01
4,4,104.378,107.662,"Yes, so welcome everyone to Zimbabwe.","[{'word': 'Yes,', 'start': 104.378, 'end': 104...",SPEAKER_03
...,...,...,...,...,...,...
1923,1923,7730.415,7730.776,No.,"[{'word': 'No.', 'start': 7730.415, 'end': 773...",NaN
1924,1924,7734.939,7739.043,"Okay, it's time now, and nobody's ever said th...","[{'word': 'Okay,', 'start': 7734.939, 'end': 7...",SPEAKER_01
1925,1925,7740.344,7741.185,Save the world.,"[{'word': 'Save', 'start': 7740.344, 'end': 77...",SPEAKER_01
1926,1926,7744.188,7747.271,We don't all have to go to the same old folks'...,"[{'word': 'We', 'start': 7744.188, 'end': 7744...",SPEAKER_01


### Import all the libraries required


In [3]:
import numpy as np
import json
import glob
import re

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Cant remember why needed this ... 
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


## Tidy Preprocessing! 

In [8]:

from nltk.corpus.util import LazyCorpusLoader
from nltk.corpus import stopwords as nltk_stopwords

# 1. Remove emails, newline characters, and non-alphabetic characters
preprocessed_df['cleaned_text'] = preprocessed_df['text'].str.replace(r'\S+@\S+', '', regex=True)
preprocessed_df['cleaned_text'] = preprocessed_df['cleaned_text'].str.replace(r'http\S+|www\S+', '', regex=True)
preprocessed_df['cleaned_text'] = preprocessed_df['cleaned_text'].str.replace(r'\n', ' ', regex=True)
preprocessed_df['cleaned_text'] = preprocessed_df['cleaned_text'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# 2. Convert to lowercase
preprocessed_df['cleaned_text'] = preprocessed_df['cleaned_text'].str.lower()

# 3. Tokenize the text
preprocessed_df['tokens'] = preprocessed_df['cleaned_text'].apply(lambda x: gensim.utils.simple_preprocess(x, deacc=True))

# 4. Remove stopwords
stopwords = set(nltk_stopwords.words('english'))
preprocessed_df['tokens'] = preprocessed_df['tokens'].apply(lambda x: [word for word in x if word not in stopwords])

# 5. Lemmatize the tokens
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
preprocessed_df['lemmatized_tokens'] = preprocessed_df['tokens'].apply(lambda x: [token.lemma_ for token in nlp(" ".join(x)) if token.pos_ in ["NOUN", "ADJ", "VERB", "ADV"]])

# 6. Remove tokens that are less than 3 characters long
preprocessed_df['lemmatized_tokens'] = preprocessed_df['lemmatized_tokens'].apply(lambda x: [word for word in x if len(word) >= 3])

# 7. Remove rows where the list length in lemmatized_tokens is 0
preprocessed_df = preprocessed_df[preprocessed_df['lemmatized_tokens'].apply(len) > 0]

#  Tested each step with a few preprocessed_df.iloc[6]


In [9]:
# Display the preprocessed dataframe
preprocessed_df.head()


,Unnamed: 0,start,end,text,words,speaker,cleaned_text,tokens,lemmatized_tokens
0,0,63.240,66.422,Hello and welcome to our last ever Grand Tour.,"[{'word': 'Hello', 'start': 63.24, 'end': 63.4...",SPEAKER_01,hello and welcome to our last ever grand tour,"[hello, welcome, last, ever, grand, tour]","[last, ever, grand, tour]"
1,1,67.182,74.707,And because this is the last time that James R...,"[{'word': 'And', 'start': 67.182, 'end': 67.28...",SPEAKER_01,and because this is the last time that james r...,"[last, time, james, richard, ever, work, toget...","[last, time, ever, work, together, say, import..."
2,2,75.507,84.852,He said we should each buy a medium-sized elec...,"[{'word': 'He', 'start': 75.507, 'end': 75.687...",SPEAKER_01,he said we should each buy a mediumsized elect...,"[said, buy, mediumsized, electric, car, see, m...","[say, buy, mediumsize, electric, car, see, man..."
3,3,86.393,89.415,And we decided to ignore him and come here.,"[{'word': 'And', 'start': 86.393, 'end': 86.47...",SPEAKER_01,and we decided to ignore him and come here,"[decided, ignore, come]","[decide, ignore, come]"
4,4,104.378,107.662,"Yes, so welcome everyone to Zimbabwe.","[{'word': 'Yes,', 'start': 104.378, 'end': 104...",SPEAKER_03,yes so welcome everyone to zimbabwe,"[yes, welcome, everyone, zimbabwe]",[zimbabwe]


### Export the Pre Processed DF to CSV

* removed emails
* converted to lower
* remove stop words
* tokenized and lemmatize
* remove tokens <3 char
* remove null dictionaries (rows)

In [10]:
## Double check file names 2007 or 2024
preprocessed_df.to_csv("./data/06.preprocess_completed_2024.csv")

### Split the data frame into 3 (one per presenter)

In [ ]:
## 2007
# Create a new DataFrame for each speaker
May_df = preprocessed_df[preprocessed_df['speaker'] == 'May']
Clarkson_df = preprocessed_df[preprocessed_df['speaker'] == 'Clarkson']
Hammond_df = preprocessed_df[preprocessed_df['speaker'] == 'Hammond']

# Display the first few rows of each DataFrame (optional)
# May_df.head()

Clarkson_df.head()

# Hammond_df.head()

,Unnamed: 0,start,end,text,words,speaker,cleaned_text,tokens,lemmatized_tokens
1,1,23.108,24.029,Thank you very much.,"[{'word': 'Thank', 'start': 23.108, 'end': 23....",Clarkson,thank you very much,"[thank, much]","[thank, much]"
2,2,24.049,34.256,"Now, as you know, the producers on this show l...","[{'word': 'Now,', 'start': 24.049, 'end': 24.1...",Clarkson,now as you know the producers on this show lik...,"[know, producers, show, like, give, us, challe...","[know, producer, show, give, challenge, specif..."
3,3,34.597,39.640,Then they set unbelievably hard tasks to do to...,"[{'word': 'Then', 'start': 34.597, 'end': 34.7...",Clarkson,then they set unbelievably hard tasks to do to...,"[set, unbelievably, hard, tasks, see, one, us,...","[set, unbelievably, hard, task, see, get, good..."
16,16,100.860,102.060,It is a Lancia Beta.,"[{'word': 'It', 'start': 100.86, 'end': 100.96...",Clarkson,it is a lancia beta,"[lancia, beta]",[beta]
17,17,102.220,102.440,"Coupé, 1981.","[{'word': 'Coupé,', 'start': 102.22, 'end': 10...",Clarkson,coup,[coup],[coup]


### Convert the DF to a LIST

In [ ]:
### 2007 Botswana

Clarkson_cleaned_texts = Clarkson_df["lemmatized_tokens"].to_list()
May_cleaned_texts = May_df["lemmatized_tokens"].to_list()
Hammond_cleaned_texts = Hammond_df["lemmatized_tokens"].to_list()	


print(Hammond_cleaned_texts)

[['week', 'top', 'gear', 'special', 'come', 'real', 'humdinger'], ['give', 'quid', 'tell', 'buy', 'car'], ['hell', 'man'], ['much', 'well', 'nick'], ['much', 'change', 'buy', 'many', 'bean'], ['much', 'simple', 'get'], ['get', 'move', 'part', 'year'], ['horsepower'], ['sport', 'version', 'want', 'lairy'], ['bit', 'thick'], ['sorry'], ['think', 'car', 'inspire', 'latterly', 'build', 'become', 'rubbish'], ['course', 'indirectly'], ['quite', 'lot', 'reasonably', 'average', 'car', 'owe'], ['brake', 'terrible', 'work', 'wheel'], ['work', 'well', 'wheel', 'wheel'], ['overtake', 'truck'], ['pull', 'slipstream'], ['really', 'come'], ['still', 'truck'], ['still', 'truck'], ['happy', 'car', 'world'], ['call', 'oliver'], ['ever', 'name', 'car', 'top'], ['wish', 'say'], ['know'], ['horn'], ['oliver', 'get', 'cold'], ['listen'], ['fun', 'discover', 'travel', 'bill'], ['take', 'look', 'car'], ['massive'], ['knock', 'oliver'], ['knock', 'car'], ['fine', 'fine', 'call'], ['oliver', 'think', 'talk'], [

In [12]:
### 2024 Botswana

ALL_cleaned_texts = preprocessed_df["lemmatized_tokens"].to_list()

print(ALL_cleaned_texts)

[['last', 'ever', 'grand', 'tour'], ['last', 'time', 'ever', 'work', 'together', 'say', 'important'], ['say', 'buy', 'mediumsize', 'electric', 'car', 'see', 'many', 'lap', 'charge'], ['decide', 'ignore', 'come'], ['zimbabwe'], ['drive', 'crummy', 'electric', 'car', 'either'], ['instead', 'decide', 'simply', 'buy', 'thing', 'always', 'want'], ['buy'], ['always', 'love', 'stag'], ['love', 'small', 'boy'], ['never', 'actually', 'think'], ['honest', 'think', 'allow'], ['people', 'stag'], ['never', 'even', 'drive', 'accordance', 'rule', 'never', 'meet', 'hero'], ['delighted', 'say', 'absolutely', 'lovely'], ['point', 'lewis', 'collin', 'arrive'], ['proud', 'owner', 'threelitre'], ['get'], ['well'], ['really'], ['chance', 'argue', 'arrive'], ['already', 'make', 'mistake'], ['look'], ['buy'], ['year', 'ago', 'use', 'drive', 'prove', 'ideal', 'car', 'africa'], ['break', 'time'], ['front', 'engine'], ['midengine'], ['make', 'several', 'modification', 'suit', 'condition'], ['let', 'talk'], ['bac

## Step 3 - Create the Corpora, Dictionary & LDA

There are 3 data frames, so create ensure consistent and efficient treatment, create a function to pass each df through the same series of steps. These include: 

1. create the dictionary of terms
2. create the corpus (count of each dictionary term)
3. create the LDA Model
4. run the LDA model

In [13]:
def create_corpora_and_LDA(data_sets, names, num_topics = 5):
    results_dict = {}
    for data, name in zip(data_sets, names):
        # STEP 1 - Create dictionary
        id2word = corpora.Dictionary(data)
        print(f"Dictionary for {name}:")
        print(id2word)

        # STEP 2 - Create Corpus
        texts = data

        # STEP 3 - Term Document Frequency
        corpus = [id2word.doc2bow(text) for text in texts]

        print(f"Corpus for {name}:")
        print(corpus)

        # STEP 4- Create LDA Model
        lda_model = gensim.models.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                           random_state=100,
                                           chunksize=200,
                                           passes=10,
                                           per_word_topics=True)

        print(f"LDA Model for {name}:")
        print(lda_model)

        # STEP 5 - Store in dictionary
        results_dict[name] = {
            'dictionary': id2word,
            'corpus': corpus,
            'lda_model': lda_model
        }

    return results_dict

# RESULTS_DICTIONARY - Create a dictionary of the data sets, their respective names, and the info created from the above function. 


In [ ]:
### 2007 Botswana
data_sets = [May_cleaned_texts, Clarkson_cleaned_texts, Hammond_cleaned_texts]
names = ['May', 'Clarkson', 'Hammond']
results_dict = create_corpora_and_LDA(data_sets, names)

In [14]:
### 2024 Botswana
data_sets = [ALL_cleaned_texts]
names = ['ALL']
results_dict_2024 = create_corpora_and_LDA(data_sets, names)

Dictionary for ALL:
Dictionary<1533 unique tokens: ['ever', 'grand', 'last', 'tour', 'important']...>
Corpus for ALL:
[[(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (2, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)], [(5, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(17, 1), (18, 1), (19, 1)], [(20, 1)], [(10, 1), (12, 1), (21, 1), (22, 1), (23, 1)], [(9, 1), (18, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)], [(9, 1)], [(24, 1), (29, 1), (30, 1)], [(29, 1), (31, 1), (32, 1)], [(33, 1), (34, 1), (35, 1)], [(35, 1), (36, 1), (37, 1)], [(30, 1), (38, 1)], [(22, 1), (34, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1)], [(5, 1), (44, 1), (45, 1), (46, 1)], [(47, 1), (48, 1), (49, 1), (50, 1)], [(51, 1), (52, 1), (53, 1)], [(54, 1)], [(55, 1)], [(56, 1)], [(47, 1), (57, 1), (58, 1)], [(59, 1), (60, 1), (61, 1)], [(62, 1)], [(9, 1)], [(10, 1), (22, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)], [(6, 1), (69, 1)], [(70, 1), (71, 1)], [(72, 1)], [(60, 

In [ ]:
### 2007

#Print the keyword in the 5 topics
# 
import pprint

for name, result in results_dict.items():
    print(f"Results for {name}:")
    pprint.pprint(result['lda_model'].print_topics())


# print(lda_model.print_topics())
# doc_lda = lda_model[corpus]

#We created 5 topics. You can see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics() as shown below:

In [ ]:
### 2024

## Change is the results_dict_2024

import pprint

for name, result in results_dict_2024.items():
    print(f"Results for {name}:")
    pprint.pprint(result['lda_model'].print_topics())

Results for ALL:
[(0,
  '0.026*"time" + 0.024*"mile" + 0.023*"last" + 0.018*"little" + 0.017*"bit" + '
  '0.016*"end" + 0.016*"use" + 0.015*"first" + 0.013*"door" + 0.013*"ever"'),
 (1,
  '0.045*"car" + 0.036*"come" + 0.034*"good" + 0.021*"know" + 0.019*"mean" + '
  '0.014*"much" + 0.012*"never" + 0.012*"railway" + 0.012*"set" + '
  '0.012*"world"'),
 (2,
  '0.070*"get" + 0.045*"think" + 0.042*"look" + 0.026*"see" + 0.021*"right" + '
  '0.020*"wheel" + 0.017*"thing" + 0.016*"car" + 0.013*"old" + 0.012*"need"'),
 (3,
  '0.036*"say" + 0.029*"thank" + 0.028*"make" + 0.025*"really" + 0.022*"take" '
  '+ 0.016*"love" + 0.014*"buy" + 0.014*"want" + 0.012*"stop" + 0.012*"big"'),
 (4,
  '0.028*"way" + 0.022*"road" + 0.021*"drive" + 0.021*"well" + 0.020*"back" + '
  '0.017*"actually" + 0.016*"quite" + 0.015*"turn" + 0.013*"work" + '
  '0.013*"ill"')]


## Visualize LDA Topic Models for each Presenter

### Create a Visualize function

In [ ]:
### 2007

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

def visualize_lda_model(results_dict, dataset_name):
    # Access the dictionary, corpus, and LDA model for the specified dataset
    dictionary = results_dict[dataset_name]['dictionary']
    corpus = results_dict[dataset_name]['corpus']
    lda_model = results_dict[dataset_name]['lda_model']

    # Prepare the visualization
    vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds='mmds', R=30)

    # Display the visualization
    return vis


In [19]:
### 2024

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

def visualize_lda_model(results_dict, dataset_name):
    # Access the dictionary, corpus, and LDA model for the specified dataset
    dictionary = results_dict_2024[dataset_name]['dictionary']
    corpus = results_dict_2024[dataset_name]['corpus']
    lda_model = results_dict_2024[dataset_name]['lda_model']

    # Prepare the visualization
    vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds='mmds', R=30)

    # Display the visualization
    return vis

### Call the LDA Visualization for a given Presenter

In [ ]:
### 2007

# Visualize the LDA model for 'X'
vis = visualize_lda_model(results_dict, 'Clarkson')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.064187  0.180421       1        1  24.769145
2     -0.081196 -0.194077       2        1  21.362234
4     -0.178644  0.004536       3        1  18.774905
3      0.149883 -0.123811       4        1  17.997135
0      0.174146  0.132931       5        1  17.096580, topic_info=        Term       Freq      Total Category  logprob  loglift
16       get  25.000000  25.000000  Default  30.0000  30.0000
66     right  17.000000  17.000000  Default  29.0000  29.0000
466  replace  13.000000  13.000000  Default  28.0000  28.0000
59     think  14.000000  14.000000  Default  27.0000  27.0000
107      top  13.000000  13.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
416    stage   0.862693   1.943257   Topic5  -5.5645   0.9542
7       know   1.147563   4.785494   Topic5  -5.2792   0.3383
83      make   1.394744  10.660455   Topic5  -5.0841  -0.2675
162      bad   0.916030   2.610727   Topic5  -5.5045   0.7190
19       see   0.832627   4.936446   Topic5  -5.6000  -0.0135

[220 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
284       4  0.984718  absolutely
641       2  0.959078    actually
285       4  0.552626     amazing
214       4  0.893870        away
383       1  0.912701        back
...     ...       ...         ...
173       5  0.764107        wide
200       4  0.929384        work
133       1  0.519853       world
353       3  0.852807        year
65        3  0.528651    zimbabwe

[210 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 5, 4, 1])

In [ ]:
### 2024

# Visualize the LDA model for 'ALL'
vis_2024 = visualize_lda_model(results_dict_2024, 'ALL')
vis_2024

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.087069  0.240128       1        1  20.699143
2     -0.068632 -0.241105       2        1  20.510478
1     -0.246919 -0.009870       3        1  20.302769
4      0.191466 -0.137787       4        1  19.943406
0      0.211154  0.148634       5        1  18.544204, topic_info=      Term       Freq      Total Category  logprob  loglift
54     get  78.000000  78.000000  Default  30.0000  30.0000
35   think  44.000000  44.000000  Default  29.0000  29.0000
62    look  41.000000  41.000000  Default  28.0000  28.0000
10     car  59.000000  59.000000  Default  27.0000  27.0000
17    come  35.000000  35.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
188  first  13.722328  17.435391   Topic5  -4.1688   1.4455
40    even   7.522151   9.747042   Topic5  -4.7700   1.4259
433  start   7.813199  11.354107   Topic5  -4.7320   1.3112
27   thing   5.021569  22.415905   Topic5  -5.1741   0.1890
123    put   4.554910  12.370652   Topic5  -5.2716   0.6859

[210 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
44        1  0.844583  absolutely
33        2  0.153960    actually
33        4  0.821120    actually
24        1  0.911197      always
1442      3  0.984843     amazing
...     ...       ...         ...
244       3  0.920820       world
412       2  0.984946       wrong
68        2  0.330852        year
68        3  0.661705        year
20        2  0.823950    zimbabwe

[198 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 5, 1])

### HTML for Presenters together

In [ ]:
# Extract the Variables from the results dictionary and declare new variables (copy n paste method)
'''

# Extract the variables for May
lda_model_May = results_dict['May']['lda_model']
corpus_May = results_dict['May']['corpus']
dictionary_May = results_dict['May']['dictionary']

lda_model_Hammond = results_dict['Hammond']['lda_model']
corpus_Hammond = results_dict['Hammond']['corpus']
dictionary_Hammond = results_dict['Hammond']['dictionary']

# Similarly, you can extract and visualize for Clarkson and Hammond
lda_model_Clarkson = results_dict['Clarkson']['lda_model']
corpus_Clarkson = results_dict['Clarkson']['corpus']
dictionary_Clarkson = results_dict['Clarkson']['dictionary']

'''

### Function to generate and save PyLDAvis visualization

In [ ]:
### 2007

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

def generate_and_save_vis(results_dict, name):
    lda_model = results_dict[name]['lda_model']
    corpus = results_dict[name]['corpus']
    dictionary = results_dict[name]['dictionary']
    vis = gensimvis.prepare(lda_model, corpus, dictionary)
    pyLDAvis.save_html(vis, f'./outputs/vis_{name}.html')

# Generate and save visualizations for each dataset
for name in names:
    generate_and_save_vis(results_dict, name)

# Combine the HTML files into a single HTML file
with open('combined_vis.html', 'w') as outfile:
    for name in names:
        with open(f'./outputs/vis_{name}.html') as infile:
            outfile.write(infile.read())

In [23]:
### 2024 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

def generate_and_save_vis(results_dict, name):
    lda_model = results_dict_2024[name]['lda_model']
    corpus = results_dict_2024[name]['corpus']
    dictionary = results_dict_2024[name]['dictionary']
    vis = gensimvis.prepare(lda_model, corpus, dictionary)
    pyLDAvis.save_html(vis, f'./outputs/vis_{name}_2024.html')

# Generate and save visualizations for each dataset
for name in names:
    generate_and_save_vis(results_dict_2024, name)

# # Combine the HTML files into a single HTML file
# with open('combined_vis.html', 'w') as outfile:
#     for name in names:
#         with open(f'./outputs/vis_{name}.html') as infile:
#             outfile.write(infile.read())

In [ ]:
### Dont need this any more as created above function.

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Generate PyLDAvis visualizations
vis_May = gensimvis.prepare(lda_model_May, corpus_May, dictionary_May)
vis_Clarkson = gensimvis.prepare(lda_model_Clarkson, corpus_Clarkson, dictionary_Clarkson)
vis_Hammond = gensimvis.prepare(lda_model_Hammond, corpus_Hammond, dictionary_Hammond)

# Save each visualization as an HTML file
pyLDAvis.save_html(vis_May, './outputs/vis_May.html')
pyLDAvis.save_html(vis_Clarkson, './outputs/vis_Clarkson.html')
pyLDAvis.save_html(vis_Hammond, './outputs/vis_Hammond.html')

# Combine the HTML files into a single HTML file
with open('./outputs/combined_vis.html', 'w') as outfile:
    for fname in ['./outputs/vis_May.html', './outputs/vis_Clarkson.html', './outputs/vis_Hammond.html']:
        with open(fname) as infile:
            outfile.write(infile.read())

# Now you can open 'combined_vis.html' to see all three visualizations together

# Part 5: Evaluation - Coherence 

Topic coherence measures the average similarity between top words having the highest weights in a topic i.e relative distance between the top words.


In [43]:


for name in results_dict.keys():
	lda_model = results_dict[name]['lda_model']
	texts = eval(f"{name}_cleaned_texts")
	dictionary = results_dict[name]['dictionary']
	
	coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
	coherence_lda = coherence_model_lda.get_coherence()
	print(f'Coherence Score for {name}: ', coherence_lda)



Coherence Score for May:  0.6808374806112967
Coherence Score for Clarkson:  0.5909402916987949
Coherence Score for Hammond:  0.6659895742797534


# Part 6: Model Improvement - How many topics? 

## Define function to Iterate Coherence: 

In [44]:

# Define the function to iterate
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
	coherence_values = []
	model_list = []
	
	for name in names:
		dictionary = results_dict[name]['dictionary']
		corpus = results_dict[name]['corpus']
		texts = eval(f"{name}_cleaned_texts")
		
		for num_topics in range(start, limit, step):
			model = gensim.models.LdaModel(corpus=corpus, num_topics=num_topics, random_state=100, chunksize=200, passes=10, per_word_topics=True, id2word=dictionary)
			model_list.append(model)
			coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
			coherence_values.append(coherencemodel.get_coherence())
	
	return model_list, coherence_values


In [45]:

# Call the function and print the results for each name in results_dict
limit = 10
start = 2
step = 1

for name in names:
	print(f"Results for {name}:")
	model_list, coherence_values = compute_coherence_values(results_dict[name]['dictionary'], results_dict[name]['corpus'], eval(f"{name}_cleaned_texts"), limit, start, step)
	for m, cv in zip(range(start, limit, step), coherence_values):
		print(f"Num Topics = {m}, Coherence Value = {cv}")


Results for May:
Num Topics = 2, Coherence Value = 0.7166129819455946
Num Topics = 3, Coherence Value = 0.7157485205762296
Num Topics = 4, Coherence Value = 0.6874794865478038
Num Topics = 5, Coherence Value = 0.6808374806112967
Num Topics = 6, Coherence Value = 0.645575251871593
Num Topics = 7, Coherence Value = 0.584938584949688
Num Topics = 8, Coherence Value = 0.5339173988933685
Num Topics = 9, Coherence Value = 0.5369284102274318
Results for Clarkson:
Num Topics = 2, Coherence Value = 0.7166129819455946
Num Topics = 3, Coherence Value = 0.7157485205762296
Num Topics = 4, Coherence Value = 0.6874794865478038
Num Topics = 5, Coherence Value = 0.6808374806112967
Num Topics = 6, Coherence Value = 0.645575251871593
Num Topics = 7, Coherence Value = 0.584938584949688
Num Topics = 8, Coherence Value = 0.5339173988933685
Num Topics = 9, Coherence Value = 0.5369284102274318
Results for Hammond:
Num Topics = 2, Coherence Value = 0.7166129819455946
Num Topics = 3, Coherence Value = 0.7157485